In [1]:
#from games_setup import *
import init
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule, MoleculeStoichiometry
from SBMLLint.common.reaction import Reaction
from SBMLLint.games.som import SOM
from SBMLLint.common.simple_sbml import SimpleSBML

import collections
import itertools
import networkx as nx
import numpy as np
import os
import pandas as pd
#
from SBMLLint.common.stoichiometry_matrix import StoichiometryMatrix
from SBMLLint.games.mesgraph import MESGraph
from SBMLLint.games.games_pp import GAMES_PP, SOMStoichiometry, SOMReaction, TOLERANCE
from SBMLLint.games.games_report import GAMESReport, SimplifiedReaction
NULL_STR = ""
ReactionOperation = collections.namedtuple("ReactionOperation", 
    "reaction operation")

In [2]:
PREFIX = "BIOMD0000000"

def load_file_from_curated_data(num):

  format_num = format(num, '03d')
  filename = "%s%s_url.xml" % (PREFIX, format_num)
  path = os.path.join(cn.BIOMODELS_DIR, filename)
  simple = SimpleSBML()
  simple.initialize(path)
  return simple

In [3]:
simple = load_file_from_curated_data(248)

In [4]:
simple.reactions

[ATPase: ATP -> ADP; Tissue * Katpase * ATP,
 OxidativePhosphorylation: 6.00 ADP + CTtis -> 6.00 ATP; Tissue * Vmax * (CFtis / (Km + CFtis)) * ADP / (Kadp + ADP),
 ConvectionTransport:  -> CTcap; Qm * (CTart - CTcap),
 DiffusionTransport: CTcap -> CTtis; PSm * (CFcap - CFtis),
 CreatineKinase: ADP + PCr -> ATP + Cr; Tissue * (VfCK * ADP * PCr / (Kb * Kia) - VrCK * Cr * ATP / (Kiq * Kp)) / ((Kia + ADP) / Kia + ATP / Kiq + PCr / Kib + ADP * PCr / (Kb * Kia) + Cr * ATP / (Kiq * Kp))]

In [5]:
# Model 248 is for canceling error
# Model 14/167 for a type three error / echelon
# Model 7 for type 1 error
# Model 18 for type 2 error

######## examine model 628 for large echelon error ###########
simple = load_file_from_curated_data(248)
# for r in simple.reactions:
#   if r.category != cn.REACTION_BOUNDARY and r.category != cn.REACTION_1_1:
#     print(r.makeIdentifier(is_include_kinetics=False))
m = GAMES_PP(simple)
k = m.analyze(error_details=True)
print("analysis result is: ", k)
gr = GAMESReport(m)

Model analyzed...
Type I error:  []
Type II error:  []
Canceling error:  [OxidativePhosphorylation: {CTcap=CTtis} -> ]
Echelon error:  []
Type III error:  [] 

At least one error found.

analysis result is:  True


In [6]:
m.echelon_df

,statPhosphorylation,PstatDimerisation,PstatDimerisationNuc
{species_test},1.0,0.0,0.0
{Pstat_sol},1.0,-2.0,0.0
{PstatDimer_nuc=PstatDimer_sol},0.0,1.0,1.0
{Pstat_nuc=stat_nuc=stat_sol},-1.0,0.0,-2.0


In [ ]:
count = 0
for som in m.nodes:
  if len(som.molecules)>1:
    count = count + len(som.reactions)
print(count)

In [ ]:
np.linalg.cond(m.lower_inverse)

In [ ]:
len(simple.reactions)

In [ ]:
resulting_series = gr.getResultingSeries("statPhosphorylation")

In [ ]:
len(simple.reactions)

In [ ]:
type1_report, error_num = gr.reportTypeOneError([m.type_one_errors[1]], explain_details=True)
print(type1_report)
print(error_num)

In [ ]:
[m.type_three_errors[0]]

In [ ]:
a = gr.reportTypeThreeError(m.type_three_errors, explain_details=True)
a

In [ ]:
type3_report, error_num = gr.reportTypeThreeError(m.type_three_errors, explain_details=True)
print(type3_report)
print(error_num)

In [ ]:
type2_report, error_num = gr.reportTypeTwoError([m.type_two_errors[1]], explain_details=True)
print(type2_report)
print(error_num)

In [ ]:
type2_report

In [15]:
echelon_report, error_num = gr.reportEchelonError(m.echelon_errors, explain_details=False)
print(echelon_report)
print(error_num)


We detected a mass imbalance
:  -> species_test

from the following isolation set.

1. statPhosphorylation: stat_sol -> Pstat_sol + species_test
2. PstatDimerisation: 2.00 Pstat_sol -> PstatDimer_sol
3. PstatDimerisationNuc: 2.00 Pstat_nuc -> PstatDimer_nuc
4. stat_export: stat_sol -> stat_nuc
5. statDephosphorylation: Pstat_nuc -> stat_nuc
6. PstatDimer__import: PstatDimer_sol -> PstatDimer_nuc


**********************************************************************


[3]


In [5]:
canceling_report, error_num = gr.reportCancelingError(m.canceling_errors, explain_details=True)
print(canceling_report)
print(error_num)

We detected a mass imbalance
: OxidativePhosphorylation: CTtis -> 

from the followign isolation set:

1. OxidativePhosphorylation: 6.00 ADP + CTtis -> 6.00 ATP
2. ATPase: ATP -> ADP
*ATP and ADP have the same mass according to the above reaction


----------------------------------------------------------------------

----------------------------------------------------------------------



**********************************************************************


[2]


In [ ]:
data_dir=cn.BIOMODELS_DIR
#files = [f for f in os.listdir(data_dir) if f[-4:] == ".xml"]
files = [f for f in os.listdir(data_dir) if f[:7] == "BIOMD00"]
paths = [os.path.join(data_dir, filename) for filename in files]
# this will test which model has which error;;
model_count = 0
count = 0
simple = SimpleSBML
for file in files:
  try:
    simple.initialize(os.path.join(data_dir, file))
    model_count += 1
    if model_count%100 == 0:
      print("we are doing model", model_count)
    m = GAMES_PP(simple)
    if m.analyze(error_details=False):
      if (len(simple.reactions) > 100) and (len(m.type_one_errors)>1):
        print(file, "has type I error with more than 100 reactions")
        count += 1
    if count > 5:
      break
  except:
    pass

In [ ]:
# this will test which model has which error;;
model_count = 0
count = 0
for i in range(1, 150):
  try:
    simple = load_file_from_curated_data(i)
    model_count += 1
    if model_count%100 == 0:
      print("we are doing model", model_count)
    m = GAMES_PP(simple)
    m.analyze(error_details=False)
    if m.echelon_errors or m.type_three_errors:
        print("model", model_count, " has an echelon or type three error!")
        if count == 5:
          break
  except:
    pass